In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/live_109-2-1-1-2_720.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/live_108-1-1-1-2_180.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/live_110-2-1-1-1_420.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/spoof_015-1-3-1-2_60.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/spoof_015-2-3-2-1_360.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/spoof_018-2-3-4-2_300.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/spoof_018-1-3-4-1_300.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/live_109-2-1-1-1_1140.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/spoof_015-2-3-4-1_240.jpg
/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data/spoof_015-2-3-2-1_120.jpg
/kag

In [ ]:
# EfficientNet-Lite Training + Evaluation Script (Controlled & Random Stitching)

import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Sobel Filter for Edge Detection
def sobel_filter(image):
    sobel_x = torch.tensor([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=torch.float32).expand(3, 1, 3, 3)
    sobel_y = torch.tensor([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=torch.float32).expand(3, 1, 3, 3)
    sobel_x, sobel_y = sobel_x.to(image.device), sobel_y.to(image.device)
    image = image.unsqueeze(0)
    grad_x = F.conv2d(image, sobel_x, padding=1, groups=3)
    grad_y = F.conv2d(image, sobel_y, padding=1, groups=3)
    edge = torch.sqrt(grad_x ** 2 + grad_y ** 2)
    return edge.squeeze(0)

# Patch Dataset (Controlled or Random Stitching)
class PatchGridDataset(Dataset):
    def __init__(self, root_dir, transform=None, mode="controlled"):
        self.image_paths = sorted(glob.glob(os.path.join(root_dir, '*.jpg')))
        self.transform = transform
        self.mode = mode
        self.patch_size = 16
        self.grid_size = 14
        self.label_map = [1 if 'live' in os.path.basename(p).lower() else 0 for p in self.image_paths]
        self.all_images = [Image.open(p).convert('RGB') for p in self.image_paths]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        label = self.label_map[idx]
        base_img = self.all_images[idx]
        if self.transform:
            base_img = self.transform(base_img)

        patches, gt_map = [], []
        for _ in range(self.grid_size * self.grid_size):
            if self.mode == "controlled":
                if random.random() > 0.5:
                    img = base_img
                    lab = label
                else:
                    ridx = random.randint(0, len(self.all_images) - 1)
                    img = self.transform(self.all_images[ridx])
                    lab = self.label_map[ridx]
            else:
                ridx = random.randint(0, len(self.all_images) - 1)
                img = self.transform(self.all_images[ridx])
                lab = self.label_map[ridx]

            top = random.randint(0, img.shape[1] - self.patch_size)
            left = random.randint(0, img.shape[2] - self.patch_size)
            patch = img[:, top:top+self.patch_size, left:left+self.patch_size]
            edge = sobel_filter(patch)
            combined = torch.cat([patch, edge], dim=0)
            patches.append(combined)
            gt_map.append(lab)

        grid = torch.stack(patches).view(self.grid_size, self.grid_size, 6, self.patch_size, self.patch_size)
        grid = grid.permute(2, 0, 3, 1, 4).reshape(6, self.grid_size*self.patch_size, self.grid_size*self.patch_size)
        label_map = torch.tensor(gt_map, dtype=torch.float32).view(self.grid_size, self.grid_size)
        return grid, label_map

# EfficientNet-Lite based Model
class EfficientNetGrid(nn.Module):
    def __init__(self):
        super().__init__()
        weights = EfficientNet_B0_Weights.DEFAULT
        base = efficientnet_b0(weights=weights)
        base.features[0][0] = nn.Conv2d(6, 32, 3, 2, 1, bias=False)
        self.features = base.features
        self.conv1x1 = nn.Conv2d(1280, 1, 1)
        self.upsample = nn.Upsample(size=(14, 14), mode='bilinear', align_corners=False)

    def forward(self, x):
        x = self.features(x)
        x = self.conv1x1(x)
        x = self.upsample(x)
        return x.squeeze(1)

# Evaluation

def evaluate(model, dataloader, device):
    model.eval()
    y_true, y_score = [], []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            out = torch.sigmoid(out)
            y_true.extend((y.mean(dim=(1,2)) > 0.5).int().cpu().numpy())
            y_score.extend(out.mean(dim=(1,2)).cpu().numpy())

    preds = [1 if p > 0.5 else 0 for p in y_score]
    cm = confusion_matrix(y_true, preds)
    tn, fp, fn, tp = cm.ravel()
    apcer = fp / (fp + tn + 1e-6)
    bpcer = fn / (fn + tp + 1e-6)
    acer = (apcer + bpcer) / 2
    acc = 100 * (tp + tn) / (tp + tn + fp + fn)
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    return acc, apcer, bpcer, acer, fpr, tpr, roc_auc

# Training Loop

def train_model(model, loaders, device, save_prefix):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    criterion = nn.BCEWithLogitsLoss()
    train_loader, val_loader = loaders

    train_accs, val_accs, losses = [], [], []
    for epoch in range(1, 26):
        model.train()
        total_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.clamp(0,1).to(device)
            out = model(x)
            loss = criterion(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        scheduler.step()
        avg_loss = total_loss / len(train_loader)
        acc, apcer, bpcer, acer, fpr, tpr, roc_auc = evaluate(model, val_loader, device)
        print(f"Epoch {epoch}: Loss={avg_loss:.4f}, ACC={acc:.2f}%, ACER={acer:.4f}")

        losses.append(avg_loss)
        val_accs.append(acc)

    # Plot
    plt.figure(); plt.plot(val_accs); plt.title("Validation Accuracy"); plt.savefig(f"{save_prefix}_acc.png")
    plt.figure(); plt.plot(losses); plt.title("Training Loss"); plt.savefig(f"{save_prefix}_loss.png")
    plt.figure(); plt.plot(fpr, tpr, label=f"ROC (AUC={roc_auc:.2f})"); plt.legend(); plt.savefig(f"{save_prefix}_roc.png")

    return acc, apcer, bpcer, acer, model

# Run Controlled and Random

def run_all():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
    results = {}
    for mode in ['controlled', 'random']:
        print(f"\n--- {mode.upper()} Stitching ---")
        model = EfficientNetGrid().to(device)

        train_set = PatchGridDataset('/kaggle/input/siw-dataset-images/train_data-20230322T161839Z-001/train_data', transform, mode)
        val_set = PatchGridDataset('/kaggle/input/siw-dataset-images/val_data-20230322T161837Z-001/val_data', transform, mode)
        test_set = PatchGridDataset('/kaggle/input/siw-dataset-images/test_data-20230322T161835Z-001/test_data', transform, mode)

        train_loader = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_set, batch_size=8, shuffle=False, num_workers=2)
        test_loader = DataLoader(test_set, batch_size=8, shuffle=False, num_workers=2)

        acc, apcer, bpcer, acer, model = train_model(model, (train_loader, val_loader), device, save_prefix=mode+'_eff')
        test_acc, test_apcer, test_bpcer, test_acer, *_ = evaluate(model, test_loader, device)
        print(f"Test Accuracy: {test_acc:.2f}%, APCER: {test_apcer:.4f}, BPCER: {test_bpcer:.4f}, ACER: {test_acer:.4f}")
        results[mode] = dict(ACC=acc, APCER=apcer, BPCER=bpcer, ACER=acer, TestACC=test_acc, TestACER=test_acer)

    # Bar Chart
    x = np.arange(len(results))
    width = 0.12
    metrics = list(results[list(results.keys())[0]].keys())

    plt.figure(figsize=(12, 6))
    for i, metric in enumerate(metrics):
        vals = [results[m][metric] for m in results]
        plt.bar(x + i * width, vals, width=width, label=metric)
    plt.xticks(x + (len(metrics)/2)*width, list(results.keys()))
    plt.ylabel("Scores")
    plt.title("EfficientNet-Lite: Random vs Controlled")
    plt.legend()
    plt.savefig("bar_eff_comparison.png")

if __name__ == '__main__':
    run_all()



--- CONTROLLED Stitching ---
Epoch 1: Loss=0.3534, ACC=87.44%, ACER=0.1233
Epoch 2: Loss=0.3152, ACC=53.94%, ACER=0.5000
Epoch 3: Loss=0.3018, ACC=85.96%, ACER=0.1303
Epoch 4: Loss=0.2963, ACC=88.92%, ACER=0.1033
Epoch 5: Loss=0.2910, ACC=90.64%, ACER=0.1002
Epoch 6: Loss=0.2821, ACC=90.76%, ACER=0.0952
Epoch 7: Loss=0.2979, ACC=97.54%, ACER=0.0260
Epoch 8: Loss=0.2769, ACC=97.17%, ACER=0.0265
Epoch 9: Loss=0.2786, ACC=98.65%, ACER=0.0126
Epoch 10: Loss=0.2709, ACC=93.10%, ACER=0.0641
Epoch 11: Loss=0.2622, ACC=95.94%, ACER=0.0388
Epoch 12: Loss=0.2624, ACC=96.06%, ACER=0.0367
Epoch 13: Loss=0.2616, ACC=97.54%, ACER=0.0234
Epoch 14: Loss=0.2592, ACC=88.67%, ACER=0.1052
Epoch 15: Loss=0.2546, ACC=92.98%, ACER=0.0651
Epoch 16: Loss=0.2570, ACC=92.86%, ACER=0.0664
Epoch 17: Loss=0.2536, ACC=95.44%, ACER=0.0424
Epoch 18: Loss=0.2531, ACC=97.29%, ACER=0.0255
Epoch 19: Loss=0.2547, ACC=96.06%, ACER=0.0371
Epoch 20: Loss=0.2506, ACC=94.70%, ACER=0.0493
Epoch 21: Loss=0.2480, ACC=92.49%, ACER